# TensorTools Runtime Demo

Demonstrates the usage of the runtime using a simple autoencoder model.

In [ ]:
# Force matplotlib to use inline rendering
%matplotlib inline

import os
import sys

# add path to libraries for ipython
sys.path.append(os.path.expanduser("~/libs"))

import numpy as np
import tensorflow as tf
import tensortools as tt

In [ ]:
BATCH_SIZE = 24
REG_LAMBDA = 5e-4
NUM_GPUS = 2

In [ ]:
dataset_train = tt.datasets.moving_mnist.MovingMNISTTrainDataset(input_seq_length=1, target_seq_length=0)
dataset_valid = tt.datasets.moving_mnist.MovingMNISTValidDataset(input_seq_length=1, target_seq_length=0)
#dataset_test = tt.datasets.moving_mnist.MovingMNISTTestDataset(BATCH_SIZE * NUM_GPUS,
#                                                               input_seq_length=1, target_seq_length=0)

In [ ]:
dataset_train = tt.datasets.mnist.MNISTTrainDataset()
dataset_valid = tt.datasets.mnist.MNISTValidDataset()
#dataset_test = tt.datasets.moving_mnist.MovingMNISTTestDataset(BATCH_SIZE * NUM_GPUS,
#                                                               input_seq_length=1, target_seq_length=0)

In [ ]:
class SimpleAutoencoderModel(tt.model.AbstractModel):    
    def __init__(self, reg_lambda=0.0):
        super(SimpleAutoencoderModel, self).__init__(reg_lambda)
        
    @tt.utils.attr.override
    def inference(self, inputs, targets, is_training=True,
                  device_scope=None, memory_device=None):
        input_shape = inputs.get_shape()
        
        x = tf.contrib.layers.flatten(inputs)
        encoded = tt.network.fc("FC_Enc", x, 64,
                                weight_init=tf.contrib.layers.xavier_initializer(),
                                bias_init=0.0,
                                activation=tf.nn.relu,
                                device=memory_device)
        representation = encoded
        decoded = tt.network.fc("FC_Dec", representation, x.get_shape()[1],
                                weight_init=tf.contrib.layers.xavier_initializer(),
                                bias_init=0.0,
                                activation=tf.nn.sigmoid,
                                device=memory_device)

        return tf.reshape(decoded, [-1] + [int(x) for x in input_shape[1:]])
    
    @tt.utils.attr.override
    def loss(self, predictions, targets):
        return tt.loss.bce(predictions, targets)

In [ ]:
#runtime = tt.core.MultiGpuRuntime(NUM_GPUS)
runtime = tt.core.DefaultRuntime()
runtime.register_datasets(dataset_train, dataset_valid)
runtime.register_model(SimpleAutoencoderModel(reg_lambda=REG_LAMBDA))
runtime.build(is_autoencoder=True)

In [ ]:
runtime.train(BATCH_SIZE, steps=1000)

In [ ]:
runtime.validate(batch_size=50)

In [ ]:
runtime.test(batch_size=50)

In [ ]:
batch_x, _ = dataset_valid.get_batch(batch_size=4)

predictions = runtime.predict(batch_x)

tt.visualization.display_batch(batch_x)
tt.visualization.display_batch(predictions)

In [ ]:
runtime.close()